In [1]:
import numpy as np
import pandas as pd
import os
import shutil
from tqdm import tqdm

# Quality Check Dataset Combiner

In [2]:
df1 = pd.read_csv("labels - 0.csv")
df2 = pd.read_csv("labels - 1.csv")

In [3]:
# concatinating dataframe
result = pd.concat([df1, df2])
result.head()

,File,Class
0,0_s0030_0.npy,1
1,0_s0030_1.npy,0
2,1_s0030_0.npy,1
3,1_s0030_1.npy,0
4,2_s0030_0.npy,1


In [4]:
result.shape

(4000, 2)

In [5]:
# shuffling rows in dataframe
result = result.sample(frac=1).reset_index(drop=True)

In [6]:
# savig the dataframe file
result.to_csv("labels.csv", index = False)

# QC Dataset to Stroke Model Dataset

### CSV File Generation

In [7]:
ml_label = pd.read_csv("ml_labels.csv")
label = pd.read_csv("labels.csv")
df = pd.DataFrame(columns = ["File", "Label"])

In [8]:
ml_label.head()

,subject_number,completed_visit_status,group,group2,DM Non-DM STROKE
0,S0030,COMPLETED,0,0,0
1,S0044,COMPLETED,0,0,0
2,S0064,COMPLETED,0,0,0
3,S0067,COMPLETED,0,0,0
4,S0068,COMPLETED,0,0,0


In [9]:
label.head()

,File,Class
0,953_s0068_1.npy,1
1,986_s0402_1.npy,1
2,58_s0030_1.npy,0
3,439_s0064_0.npy,1
4,250_s0363_0.npy,1


In [10]:
label = label[label.Class == 1.0]
label.head()

,File,Class
0,953_s0068_1.npy,1
1,986_s0402_1.npy,1
3,439_s0064_0.npy,1
4,250_s0363_0.npy,1
6,450_s0064_1.npy,1


In [11]:
for _, row in label.iterrows():
    file_name = row["File"].split("_")[1].upper()
    num = ml_label.subject_number[ml_label.subject_number == file_name].index.tolist()[0]
    num = ml_label.iloc[num,2]
    df.loc[len(df)] = [row["File"], num]
    
df.to_csv("label_file.csv", index = False)

### Dataset Generation

In [15]:
labels = pd.read_csv("label_file.csv")
print(labels.shape)
labels.head()

(2723, 2)


,File,Label
0,953_s0068_1.npy,0
1,986_s0402_1.npy,1
2,439_s0064_0.npy,0
3,250_s0363_0.npy,1
4,450_s0064_1.npy,0


In [18]:
loc = "QC Dataset/"
dest_loc = "Stroke Dataset/"
os.makedirs(dest_loc, exist_ok = True)

for _, row in tqdm(label.iterrows()):
    shutil.copy(loc + row["File"], dest_loc + row["File"])

# saving the CSV in folder
labels.to_csv(dest_loc + "labels.csv", index = False)

2723it [00:05, 526.92it/s] 


# Balancing Low Quality Signal

In [3]:
loc = "QC Dataset/"
label = pd.read_csv(loc + "labels.csv")
print(label.shape)
label.head()

(4000, 2)


,File,Class
0,953_s0068_1.npy,1
1,986_s0402_1.npy,1
2,58_s0030_1.npy,0
3,439_s0064_0.npy,1
4,250_s0363_0.npy,1


In [6]:
label_df = label.copy()

for _, row in tqdm(label_df.iterrows()):
    if row["Class"] == 0:
        filename = row["File"].split(".")[0] + "_1.npy"
        shutil.copy(loc + row["File"], loc + filename)
        label.loc[len(label)] = [filename, row["Class"]]
label.shape

4000it [00:05, 736.94it/s] 


(5277, 2)

In [12]:
# shuffling rows in dataframe
label = label.sample(frac=1).reset_index(drop=True)

label.to_csv(loc + "labels.csv", index = False)